In [8]:
import json
import nltk
import string
import pandas as pd

from pprint import pprint
from collections import Counter
from nltk.corpus import stopwords
from utils import sentencize_text

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/matheus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def remove_stopwords(token_list: list) -> list:
    stop_words = set(stopwords.words('portuguese'))
    further_punctuation = [
        ",", ".", ")", "(", "%", "|",
        "[", "]", "=", "'", "}", ":",
        ";", "˜"
    ]
    stop_words = stop_words.union(list(string.punctuation))
    stop_words = stop_words.union(further_punctuation)

    return [w for w in token_list if not w.lower() in stop_words]

def get_token_freqs(token_list: list) -> pd.DataFrame:
    token_counts = Counter(token_list)
    return pd.DataFrame.from_dict(Counter(token_counts), orient="index")\
        .reset_index().rename(columns={'index':'Token', 0:'Freq'})\
            .sort_values(by="Freq", ascending=False, ignore_index=True)


In [11]:
CORPUS_PATH = "./corpus.json"

In [12]:
with open(CORPUS_PATH, "r", encoding="utf-8") as f:
    corpus = json.load(f)

In [13]:
total_sentencas = 0
total_tokens = 0
total_tokens_unicos = 0
tamanho_corpus = len(corpus)

for template in corpus:
    print("Título: ", template.get("titulo"))


    num_sentencas = len(sentencize_text(template.get("artigo_completo", "")))
    filtered_tokens = remove_stopwords(
        template.get("artigo_tokenizado", [])
    )
    num_tokens = len(filtered_tokens)
    num_tokens_unicos = len(set(filtered_tokens))
    token_counts = get_token_freqs(filtered_tokens)
    pos_counts = Counter(template.get("pos_tagger", []))

    print("- Tokenização:")
    print("   Número de sentenças: ", num_sentencas)
    print("   Número de tokens: ", num_tokens)
    print("   Número de tokens únicos: ", num_tokens_unicos)
    print("   Top 10 tokens:")
    pprint(token_counts.head(10))
    print("   Down 10 tokens:")
    pprint(token_counts.tail(10))
    print()
    print("- Etiquetagem POS:")
    print("   Número de substantivos comuns: ", pos_counts.get("NOUN", 0))
    print("   Número de substantivos próprios: ", pos_counts.get("PROPN", 0))
    print("   Número total de substantivos: ", pos_counts.get("NOUN", 0) + pos_counts.get("PROPN", 0))
    print("   Número de verbos: ", pos_counts.get("VERB", 0))
    print("   Número de adposições: ", pos_counts.get("ADJ", 0))
    print()
    print("-" * 50)

    total_sentencas += num_sentencas
    total_tokens += num_tokens
    total_tokens_unicos += num_tokens_unicos


Título:  Multipurpose Quantum Network Simulators: A comparative study
- Tokenização:
   Número de sentenças:  163
   Número de tokens:  1640
   Número de tokens únicos:  813
   Top 10 tokens:
         Token  Freq
0     quântica    53
1         rede    49
2            ✓    34
3  simuladores    31
4    quânticos    24
5            1    23
6    simulação    21
7    simulador    19
8        redes    16
9        disso    15
   Down 10 tokens:
               Token  Freq
803              sob     1
804           FAPESP     1
805       Foundation     1
806            Paulo     1
807             CNPQ     1
808      Tecnológico     1
809       Científico     1
810  Desenvolvimento     1
811         Nacional     1
812         Conselho     1

- Etiquetagem POS:
   Número de substantivos comuns:  726
   Número de substantivos próprios:  219
   Número total de substantivos:  945
   Número de verbos:  261
   Número de adposições:  244

--------------------------------------------------
Título:  Anális

In [14]:
print("Estatísticas Agregadas:")
print("   Número médio de sentencas: ", float(total_sentencas) / tamanho_corpus)
print("   Número médio de tokens: ", float(total_tokens) / tamanho_corpus)
print("   Número médio de tokens únicos: ", float(total_tokens_unicos) / tamanho_corpus)

Estatísticas Agregadas:
   Número médio de sentencas:  181.15
   Número médio de tokens:  1775.9
   Número médio de tokens únicos:  780.3
